# Orchestrating Custom Pipelines in Vertex AI

**Contents:**
1. Introduction
2. Pipeline Parameters and Variables
3. Building Execution Container
4. Setting up build triggers with Cloud Build
5. Setting up Pipeline Schedule

## 1.0 Introduction

This notebook can be used as templete to set up version control, automated build triggers and scheduling for Vertex AI and MLOps use cases. The services used are: Cloud Build, Cloud Scheduler, Artifact Registry, Cloud Run and GitHub.

## 2.0 Pipeline Parameters and Variables

In [80]:
#project
PROJECT_ID = "crazy-hippo-01"
LOCATION = "us-west1"
CLIENT_SERVICE_ACCOUNT_EMAIL = "edit-rights@crazy-hippo-01.iam.gserviceaccount.com"

#artifact registry
ART_REPO_NAME = "custom-earnings-pipeline"
IMAGE_NAME = "vertex-custom-training"
from datetime import datetime
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
IMAGE_TAG = TIMESTAMP
IMAGE_URI = "us-west1-docker.pkg.dev/{}/{}/{}:{}".format(PROJECT_ID, ART_REPO_NAME, IMAGE_NAME,IMAGE_TAG)

#container execution
URI = "https://earnings-prediction-job-lzltzfj6za-uw.a.run.app/"

#build trigger
REPO_NAME = "mlops-earnings-pipeline"
REPO_OWNER = "HenrikWarf"

#schedule
CRON = "0 9 * * *" #09.00 every day
SCHEDULE_NAME = "earnings-prediction-pipeline"


## 3.0 Defining Pipeline Execution Container 

In [79]:
%%writefile Dockerfile

# Specifies base image and tag
FROM gcr.io/google.com/cloudsdktool/cloud-sdk:latest
WORKDIR /root

# Copies the trainer code to the docker image.
COPY cloud-run-execution/main.py /root/main.py
COPY pipeline-code/earnings_pipeline.json /root/earnings_pipeline.json
COPY cloud-run-execution/requirements.txt /root/requirements.txt

# Installs additional packages
RUN pip3 install -r requirements.txt 

#Execute the Application
#ENTRYPOINT ["python3", "pipeline-run.py"]
CMD exec gunicorn --bind :$PORT --workers 1 --threads 8 --timeout 0 main:app

Overwriting Dockerfile


## 4.0 Setting up build tiggers with Cloud Build

In [29]:
%%writefile cloud-build.yaml

steps: 
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'us-west1-docker.pkg.dev/${PROJECT_ID}/custom-earnings-pipeline/vertex-custom-training:${BUILD_ID}', '.']
  id: 'Building Cloud Run Container - Pipeline Execution Step'

- name: 'gcr.io/cloud-builders/docker'
  args: [ 'push', 'us-west1-docker.pkg.dev/${PROJECT_ID}/custom-earnings-pipeline/vertex-custom-training:${BUILD_ID}']
  id: 'Pushing Cloud Run Container to Artifact Registrty'

- name: 'gcr.io/cloud-builders/gcloud'
  args: 
    - 'run'
    - 'deploy'
    - 'earnings-prediction-job'
    - '--region'
    - 'us-west1'
    - '--image'
    - 'us-west1-docker.pkg.dev/${PROJECT_ID}/custom-earnings-pipeline/vertex-custom-training:${BUILD_ID}'
  id: 'Deploy and run Pipeline Execution in Cloud Run'

Overwriting cloud-build.yaml


#### Set up build trigger connection to git repo

In [76]:
!gcloud beta builds triggers create github --name=$REPO_NAME\
--repo-owner=$REPO_OWNER\
--repo-name=$REPO_NAME\
--branch-pattern=".*"\
--build-config="cloud-build.yaml"

Created [https://cloudbuild.googleapis.com/v1/projects/crazy-hippo-01/locations/global/triggers/4bfd4971-abfa-4e55-84a3-e69a5173f7eb].
NAME                     CREATE_TIME                STATUS
mlops-earnings-pipeline  2022-04-09T15:36:44+00:00


#### Deleting the build trigger

In [75]:
!gcloud beta builds triggers delete mlops-earnings-pipeline

Deleted [projects/crazy-hippo-01/locations/global/triggers/mlops-earnings-pipeline].


#### Manual Trigger

In [ ]:
!gcloud builds submit --config cloud-build.yaml .

## 5.0 Setting up Pipeline Schedule

#### Set up schedule

In [85]:
!gcloud scheduler jobs create http $SCHEDULE_NAME\
--location=$LOCATION --schedule="0 */3 * * *"\
--uri=$URI --http-method=POST --oidc-service-account-email $CLIENT_SERVICE_ACCOUNT_EMAIL

#### Delete Schedule

In [84]:
!gcloud scheduler jobs delete $SCHEDULE_NAME --location=$LOCATION --quiet

Deleted job [earnings-prediction-pipeline].
